In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [5]:
from loader import load_process_split_doc, load_process_split_doc_law
from langchain.schema import Document
from langchain.vectorstores import Chroma
import os
from langchain.embeddings.openai import OpenAIEmbeddings

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

folders = ["data/tax_law1", "data/tax_law2", "data/tax_law3", "data/tax_law4", "data/tax_law5", "data/tax_law6"]

for folder in folders:
    law_files = get_file_names(folder)
    for file in law_files:
        documents=load_process_split_doc_law(file, directory_path=folder)
        vector_store.add_documents(documents)

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# load - 참고자료
for doc in doc_options:
    vector_store.add_documents(load_process_split_doc(**doc))

vector_store.persist()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_22140\1574723115.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_commu

In [15]:
len(documents)

5

In [11]:
# Retreiver 생성
retriever = vector_store.as_retriever(
    # search_type="mmr",
    search_kwargs={"k": 5}
)

In [12]:
from langchain.prompts import ChatPromptTemplate
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 
    문서에 없는 내용일 경우 모른다고 대답해주세요.
    
    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온 후 요약 정리하여 이해하게 설명한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계한다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."
    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [13]:
retriever.invoke("개별소비세법 제2조는?")

[Document(metadata={'filename': '조세특례제한법', 'source': '조세특례제한법.pdf'}, page_content='조세특례제한법 기획재정부 (조세특례제도과-연구인력개발, 투자) 044-215-4131, 4136 행정안전부 (지방세특례제도과-지방세(취득세 등)) 044-205-3860 기획재정부 (금융세제과-금융소득 관련) 044-215-4233, 4236 기획재정부 (재산세제과-양도소득세(부동산 관련 제도)) 044-215-4313, 4317 기획재정부 (조세특례제도과-중소기업, 고용, 최저한세) 044-215-4132 기획재정부 (법인세제과-법인세) 044-215-4223 기획재정부 (환경에너지세제과-개별소비세, 주세, 유류세) 044-215-4331~4334 기획재정부 (관세제도과-관세) 044-215-4412, 4417 기획재정부 (부가가치세제과(농림,외국인,제주도 관련분야)) 044-215-4323 기획재정부 (소득세제과-소득세(근로장려세제 포함)) 044-215-4211~4214, 4216~4218 기획재정부 (국제조세제도과-외국인투자(외국법인) 등) 044-215-4653 기획재정부 (부가가치세제과(농림,외국인,제주도 특례분야 제외)) 044-215-4322 기획재정부 (조세특례제도과-지방이전 지원, 유턴기업, 직접국세) 044-215-4133 제1장 총칙 제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의 공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. 제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “내국인”이란 「소득세법」에 따른 거주자 및 「법인세법」에 따른 내국법인을 말한다. 2. “과세연도”란 「소득세법」에 따른 과세기간 또는 「법인세법」에 따른 사업연도를 말한다. 3. “과세표준신고”란 「소득세법」 제70조, 제71조, 제74조 및 제110조에 따른 과세표준확

In [14]:
# print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
# retriever.invoke("법인세법 제1조 에 대해 알려줘")
print(chain.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘")
# print(chain.invoke("개별소비세법 제1조 에 대해 알려줘"))
# retriever.invoke("개별소비세법 제1조 에 대해 알려줘")

개별소비세법 제1조는 과세대상과 세율에 대해 규정하고 있습니다. 주요 내용은 다음과 같습니다:

### 개별소비세법 제1조 (과세대상과 세율)
1. **과세대상**:
   - 특정한 물품
   - 특정한 장소의 입장행위
   - 특정한 장소에서의 유흥음식행위
   - 특정한 장소에서의 영업행위

2. **세율**:
   - **물품 가격의 100분의 20**: 투전기, 오락용 사행기구, 그 밖의 오락용품, 수렵용 총포류 등
   - **기준가격을 초과하는 과세가격에 대해 100분의 20**: 보석, 귀금속 제품, 고급 시계, 고급 융단, 고급 가방 등
   - **자동차에 대한 세율**:
     - 배기량이 2천cc를 초과하는 승용자동차와 캠핑용자동차: 100분의 5
     - 배기량이 2천cc 이하인 승용자동차와 이륜자동차: 100분의 5
     - 전기승용자동차: 100분의 5
   - **연료에 따른 세율**:
     - 휘발유: 리터당 475원
     - 경유: 리터당 340원
     - 등유: 리터당 90원
     - 중유: 리터당 17원
     - 석유가스와 천연가스에 대한 세율도 구체적으로 명시됨

이 조항은 개별소비세가 어떤 물품과 행위에 부과되는지, 그리고 세율이 어떻게 적용되는지를 상세히 설명하고 있습니다.

**이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.**

추가로 궁금하신 부분이 있다면 말씀해 주세요. 예를 들어, "개별소비세의 면세 기준은 무엇인가요?" 또는 "개별소비세 신고 절차는 어떻게 되나요?"와 같은 후속 질문을 고려해 보실 수 있습니다.


[Document(metadata={'filename': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='개별소비세법 기획재정부 (환경에너지세제과) 044-215-4331, 4336 기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336 제1조(과세대상과 세율) ① 개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행 위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다. ② 개별소비세를 부과할 물품(이하 “과세물품”이라 한다)과 그 세율은 다음과 같다. 1. 다음 각 목의 물품에 대해서는 그 물품가격에 100분의 20의 세율을 적용한다. 가. 투전기(投錢機), 오락용 사행기구(射倖器具), 그 밖의 오락용품 나. 수렵용 총포류 2. 다음 각 목의 물품에 대해서는 그 물품가격 중 대통령령으로 정하는 기준가격(이하 “기준가격”이라 한다)을 초과 하는 부분의 가격(이하 이 호에서 “과세가격”이라 한다)에 해당 세율을 적용한다. 가. 다음의 물품에 대해서는 과세가격의 100분의 20 1) 보석, 진주, 별갑(鼈甲), 산호, 호 박(琥珀) 및 상아와 이를 사용한 제품(나석을 사용한 제품은 포함한다) 2) 귀금속 제품 3) 삭제 4) 고급 시계 5) 고급 융단 6) 고급 가방 나. 다음의 물품에 대해서는 과세가격의 100분의 20 1) 고급 모피와 그 제품 2) 고급 가구 3. 다음 각 목의 자동차에 대해서는 그 물품가격에 해당 세율을 적용한다. 가. 배기량이 2천시시를 초과하는 승용자동차와 캠핑용자동차: 100분의 5 나. 배기량이 2천시시 이하인 승용자동차(배기량이 1천시시 이하인 것으로서 대통령령으로 정하는 규격의 것은 제외한다)와 이륜자동차 : 100분의 5 다. 전기승용자동차(「자동차관리법」 제3조제2항에 따른 세부기준을 고려하여 대통령령으로 정하는 규격의 것은 제외한다): 100분의 5 4. 다음 각 목의 물품에 대해서는 그 수량에 해당 세율을 적용

In [8]:
print(chain.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?"))
retriever.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?")

근로소득과 사업소득이 모두 있는 경우, 연말정산과 종합소득세 신고를 통해 두 소득을 합산하여 세금을 정산해야 합니다. 여기서는 기본적인 절차를 안내드립니다.

### 1. **근로소득에 대한 연말정산**
- 근로자는 소속 회사에서 연말정산을 진행합니다. 연말정산 시 회사에 각종 공제 항목에 대한 증빙자료를 제출하여 세금을 정산합니다.

### 2. **개인사업소득에 대한 종합소득세 신고**
- 개인사업에서 발생한 소득은 매년 5월에 종합소득세 신고를 통해 신고합니다.
- 개인사업소득 3000만원은 사업소득으로 분류되며, 근로소득과 합산하여 종합소득세를 계산합니다.

### 3. **소득 합산 및 세액 계산**
- 두 소득(근로소득 4000만원 + 사업소득 3000만원)을 합산하여 총 소득을 계산합니다.
- 총 소득에서 각종 공제(기본공제, 인적공제, 특별공제 등)를 차감하여 과세표준을 산출합니다.
- 과세표준에 따라 기본세율을 적용하여 산출세액을 계산합니다.

### 4. **세액공제 및 결정세액 산출**
- 산출세액에서 자녀세액공제, 연금계좌세액공제 등 가능한 세액공제를 적용하여 최종 결정세액을 산출합니다.

### 5. **기납부세액 확인 및 납부**
- 이미 납부한 원천징수세액(근로소득세)과 비교하여 추가로 납부할 세액이 있는 경우 납부합니다. 반대로 환급받을 세액이 있는 경우 환급받습니다.

### 추가 설명
- 사업소득의 경우, 경비 처리를 통해 과세표준을 낮출 수 있으므로 관련 증빙자료를 철저히 관리하는 것이 중요합니다.
- 각종 공제 및 세액공제 항목을 최대한 활용하여 세금 부담을 줄이는 것이 좋습니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가로 궁금하신 사항이 있다면, "근로소득과 사업소득의 공제 항목은 어떤 것이 있나요?" 또는 "종합소득세 신고 시 필요한 서류는 무엇인가요?"와 같은 질문을 고려해보세요.


[Document(metadata={'filename': '연말정산_신고안내', 'source': 'data/tax_etc/연말정산_신고안내.pdf - Table 20'}, page_content='사업(근로)소득금액 4천만원 이하 4천만원~1억원 1억원 초과 공제한도 500만원 300만원 200만원'),
 Document(metadata={'filename': '연말정산_신고안내', 'source': 'data/tax_etc/연말정산_신고안내.pdf'}, page_content='Ⅰ. 사업소득 연말정산 339 09 사업소득 원천징수영수증(연말정산용) 작성박영영(750701-1******)은 방문판매원으로 배우자(김선이, 연간 소득금액 없음), 자녀 2명(박서영 만 11세, 박현영만 10세)과 함께 살고 있음 ① 사업소득 수입금액：70,000,000원 - OO(주)로부터 2024년 방문판매수당 7천만원 발생 (OO(주)가 7천만원에 대해 2,100,000원을 원천징수 후 수당 지급) ＊방문판매에서 발생하는 사업소득 외에 타 소득이 없고, 간편장부대상자에 해당함 ②사업소득금액：20,500,000원 40,000천원 × (1 - 0.75) + 30,000천원 × (1 - 0.65) = 20,500천원 ③ 종합소득공제：6,000,000원 - 기본공제 4명(본인, 배우자, 자녀 2명) 6,000천원 ④ 그 밖의 소득공제：720,000원 - 개인연금저축 소득공제 720천원(개인연금저축 납입액 1,800천원 × 40%) ⑤ 과세표준：13,780,000원 ⑥ 산출세액：826,800원(과세표준 14,000천원 이하, 과세표준 × 6%) ⑦ 세액공제：420,000원 - 자녀세액공제(자녀 2명) 350,000원 - 표준세액공제 70,000원 ⑧ 결정세액：406,800원 ⑨ 기납부세액：2,100,000원 ⑩ 차감납부할 세액：△1,693,200원'),
 Document(metadata={'filename': '연말정산_신고안내', 'source': 'data/tax_etc/연말정산_

In [11]:
print(chain.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?"))
retriever.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?")

농업 분야에서 법인 형태로 소득을 벌어들이는 경우, 대한민국의 세법에서는 다양한 세금 혜택을 제공하고 있습니다. 특히, 조세특례제한법에서는 농업 관련 법인에 대한 세제 지원을 규정하고 있습니다. 다음은 몇 가지 주요 혜택입니다:

1. **법인세 감면**: 농업 법인은 일정 요건을 충족할 경우 법인세를 감면받을 수 있습니다. 이는 주로 농업 생산이나 가공, 유통을 목적으로 하는 법인에 적용됩니다.

2. **소득세 감면**: 농업 법인이 농작물을 직접 생산하여 판매하는 경우, 일정 부분의 소득에 대해 소득세를 감면받을 수 있습니다.

3. **부가가치세 면제**: 직접 생산한 농산물의 판매에 대해서는 부가가치세가 면제될 수 있습니다.

4. **기타 세제 혜택**: 농업 법인은 특정 기계나 설비에 대한 투자에 대해 세액 공제나 감면을 받을 수 있으며, 농업 관련 연구 개발에 대한 세제 지원도 받을 수 있습니다.

세제 혜택을 받기 위해서는 각 혜택별로 정해진 요건을 충족해야 하며, 해당 혜택이 언제까지 유효한지는 법령에 따라 달라질 수 있습니다. 구체적인 혜택을 받기 위해서는 전문 세무사나 회계사의 자문을 받는 것이 좋습니다.

"이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."

추가적으로 궁금하신 점이 있다면, "농업 법인이 받을 수 있는 구체적인 세액 공제는 어떤 것이 있나요?" 또는 "농업 법인 설립 시 주의해야 할 세법 요건은 무엇인가요?"와 같은 질문을 고려하실 수 있습니다.


[Document(metadata={'filename': '조세특례제한법', 'source': '조세특례제한법.pdf'}, page_content='말한다)은 해당 사업연도의 소득금액을 계 산할 때 손금에 산입한다. ③ 제1항에 따라 법인이 주주등으로부터 주식등을 무상으로 받아 소각함으로써 해당 법인의 다른 주주등이 얻는 이 익에 대해서는 「상속세 및 증여세법」에 따른 증여 또는 「법인세법」에 따른 익금으로 보지 아니한다. 다만, 주식등 을 증여한 주주등의 특수관계인에 대해서는 그러하지 아니하다. ④ 제1항부터 제3항까지의 규정을 적용할 때 재무구조개선계획의 내용 및 승인기준, 특수관계인의 범위, 세액 감면 의 신청, 그 밖에 필요한 사항은 대통령령으로 정한다. 제45조의2(공공기관의 구조개편을 위한 분할에 대한 과세특례) 「공공기관의 운영에 관한 법률」 제4조에 따라 공공기 관으로 지정된 내국법인(이하 “공공기관”이라 한다)이 민영화 등의 구조개편을 위하여 2010년 12월 31일까지 「상법 」 제530조의2부터 제530조의11까지의 규정에 따라 대통령령으로 정하는 분할을 하는 경우로서 그 분할이 대통령령 으로 정하는 요건을 갖춘 경우에는 「법인세법」 제46조제1항 각 호의 요건을 갖춘 분할로 보아 이 법과 「법인세법」 및 「부가가치세법」의 분할에 관한 규정을 적용한다. 제46조(기업 간 주식등의 교환에 대한 과세특례) ① 내국법인(이하 이 조에서 “교환대상법인”이라 한다)의 지배주주ㆍ 출자자 및 그 특수관계인(이하 이 조에서 “지배주주등”이라 한다)이 2017년 12월 31일 이전에 대통령령으로 정하는 재무구조개선계획(대통령령으로 정하는 자가 승인한 것에 한정한다. 이하 이 조에서 “재무구조개선계획”이라 한다 )에 따라 그 소유 주식 또는 출자지분(이하 이 조에서 “주식등”이라 한다) 전부를 양도하고 교환대상법인의 대통령령 으로 정하는 특수관계인이 아닌 다른 내국법인(이하 이 조에서 “교환양수법인”이라 한다)의 주식등을 다음 각 호의 어느 하나에 해당하는 